In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path

import sys
sys.path.append("../src")

from data_prep import *
from make_cuts import *
from fitting_functions import *
from pulse_profile_generator import *
from plot_maker import *

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
c_folder="../../data/calibration/"
d_folder="../../data/runData/"
r_n=307
Path("../../plots").mkdir(parents=True, exist_ok=True)
Path("../../plots/run_"+str(r_n)).mkdir(parents=True, exist_ok=True)
p_folder="../../plots/run_"+str(r_n)+"/"

In [4]:
%%time
p,m,r=import_data(c_folder,d_folder, 305, is_it_pulsed=False)

CPU times: user 7.38 s, sys: 1.96 s, total: 9.35 s
Wall time: 9.36 s


In [5]:
%%time
r=select_bars_with_data(r,p, subtract_pedestal=True, is_it_pulsed=False) #this needs to be run as it's own cell

Number of events: 47351


Initial pedestal-based selection performed.
Events left: 46271


CPU times: user 962 ms, sys: 418 ms, total: 1.38 s
Wall time: 1.38 s


In [8]:
# %%time # make shower profiles
# make_shower_profiles(r, m, r_n, p_folder)

In [9]:
# %%time # make the pulse profile dataframe
# pulses=make_pulse_profiles(r)
# pulses.to_csv(d_folder+"/run_"+str(r_n)+"_pulse_problems.csv")